In [1]:
# P2047

In [2]:
import json
import pathlib
import pydash
import requests
import time

def pull_duration(link):

    ''' Pull duration data from OFLC. '''

    time.sleep(4)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0'} 
    r = requests.get(link, headers=headers).text
    duration = r.split('<div class="field__label">Duration</div>')[1].split('</div>')[0]
    duration = duration.replace('<div class="field__item">','').replace('minutes','').strip()
    return duration

with open(pathlib.Path.cwd().parents[0] / 'directory.json') as direct:
    direct = json.load(direct)

for x in direct.keys():
    if pydash.get(direct, f'{x}.oflc'):
        oflc_link = 'https://www.classification.gov.au/'+pydash.get(direct, f'{x}.oflc')
        duration = pull_duration(oflc_link)
        print(x, 'P2047', duration)

Q1421747 P2047 118
Q12126942 P2047 97
Q3415693 P2047 107
